In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers

In [3]:
from utils import random_split_train_val

In [4]:
train_dataset = np.loadtxt("data/khasanshin/THRESHOLD_ALL.txt", delimiter=",")

X = train_dataset[:,0:450]
Y = train_dataset[:,600]

# 2D Convolution Net 

In [5]:
X_train, y_train, X_val, y_val = random_split_train_val(X, Y, 212)
X_train = X_train.reshape(-1, 3, 150, 1)
X_val = X_val.reshape(-1, 3, 150, 1)

In [6]:
num_classes = 5
lrs = [0.1 ** (n*0.5 + 1) for n in range(10)]
momentums = [0.8, 0.9, 0.95, 0.99]
batch_sizes = [16*n for n in range(1, 7)]
n_filters = [8*n for n in range(1, 16)]
kernel_sizes = [4*n for n in range(1, 16)]

In [7]:
def train_model(lr, momentum, batch_size, n_filters_1, n_filters_2, kernel_size):
    
    my_conv_model = Sequential([
        Input(X_train.shape[1:]),
        Conv2D(filters=n_filters_1, kernel_size=(2, kernel_size)),
        BatchNormalization(),
        ReLU(),

        Conv2D(filters=n_filters_2, kernel_size=2),
        BatchNormalization(),
        ReLU(),

        GlobalAveragePooling2D(),
        Dense(num_classes, activation='sigmoid')
    ])

    optim = optimizers.SGD(lr, momentum)
    
    my_conv_model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer=optim, metrics=['accuracy'])

    my_conv_history = my_conv_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        epochs=64, batch_size=batch_size, verbose=0)
    tic = time.time()
    
    results = my_conv_model.evaluate(X_val, y_val, batch_size=1)
    
    tac = time.time()
    
    return my_conv_history.history['val_accuracy'][-1], tac - tic

Here we use random search to find best hyperparameters

In [8]:
import pandas as pd
from random import choice
import time

In [9]:
# Long, Long grid search
row_list = []

for i in range(1024):
    lr = choice(lrs)
    momentum = choice(momentums)
    batch_size = choice(batch_sizes)
    n1 = choice(n_filters)
    n2 = choice(n_filters)
    ks = choice(kernel_sizes)
    
    val_acc, t = train_model(lr, momentum, batch_size, n1, n2, ks)
    dict_i = {'lr':lr, 'momentum':momentum, 'batch_size':batch_size, 
              'n1':n1, 'n2':n2, 'ks':ks,
              'val_acc':val_acc, 'predict_time':t}
    
    row_list.append(dict_i)

df = pd.DataFrame(row_list)
df.to_csv('data/khasanshin/Random_search.csv')

212/212 [==============================] - 0s 1ms/step - loss: 0.1485 - accuracy: 0.9434


KeyboardInterrupt: 

In [ ]:
df = pd.read_csv('data/khasanshin/Random_search.csv')

In [ ]:
df.sort_values('val_acc', ascending=False).head()